In [1]:
import pandas as pd
import sys
import numpy as np

In [2]:
df_node = pd.read_csv("train/node.csv", dtype=str)
df_relation = pd.read_csv("train/relation.csv", dtype=str)
df_triple = pd.read_csv("train/triple.csv", dtype=str)

In [7]:
triple_list = df_triple.values.tolist()
triple_count = len(df_triple)
print("triple count: {}".format(triple_count))

triple count: 262019


In [16]:
seed = np.arange(triple_count)
np.random.shuffle(seed)

train_count = int(triple_count * 0.9)
valid_count = int(triple_count * 0.05)
train_index = seed[:train_count].tolist()
valid_index = seed[train_count:train_count+valid_count].tolist()
test_index = seed[train_count+valid_count:].tolist()

In [17]:
with open("train/dglke_train.tsv", "w+") as f:
    for index in train_index:
        f.writelines("{}\t{}\t{}\n".format(triple_list[index][0], triple_list[index][1], triple_list[index][2]))
        
with open("train/dglke_valid.tsv", "w+") as f:
    for index in valid_index:
        f.writelines("{}\t{}\t{}\n".format(triple_list[index][0], triple_list[index][1], triple_list[index][2]))
        
with open("train/dglke_test.tsv", "w+") as f:
    for index in test_index:
        f.writelines("{}\t{}\t{}\n".format(triple_list[index][0], triple_list[index][1], triple_list[index][2]))


dglke_train --dataset pgkb --data_path ./train --data_files dglke_train.tsv dglke_valid.tsv dglke_test.tsv --format raw_udd_hrt --model_name TransE_l2 --batch_size 2048 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --num_proc 1 --neg_sample_size_eval 10000 --async_update

In [2]:
node_embedding = np.load("ckpts/TransE_l2_pgkb_5/pgkb_TransE_l2_entity.npy")
relation_embedding = np.load("ckpts/TransE_l2_pgkb_5/pgkb_TransE_l2_relation.npy")

In [3]:
print(node_embedding.shape)
print(relation_embedding.shape)

(81510, 400)
(14, 400)


In [4]:
node_embedding

array([[-0.25426614,  0.70644957, -0.27191836, ...,  0.89812595,
         0.46692854,  0.32273653],
       [-0.05132748,  0.07197798, -0.34075344, ...,  0.52072155,
        -0.11162469, -0.46548358],
       [ 0.31897926, -0.68682486, -0.59711933, ..., -0.62291443,
         0.09369897,  0.42894498],
       ...,
       [ 0.5190772 ,  0.36109135, -0.49053636, ...,  0.39061296,
        -0.41722012,  0.41264433],
       [-0.4263142 ,  0.23674169, -0.5240198 , ..., -0.51616716,
         0.4771007 , -0.53949714],
       [-0.33942884, -0.53039414, -0.5396247 , ..., -0.5277652 ,
         0.4809796 , -0.5057551 ]], dtype=float32)

In [5]:
df_entity_idx = pd.read_csv("train/entities.tsv", sep="\t", header=None, names=["idx", "entity_id"], dtype=str).fillna("")
df_node = pd.read_csv("train/node.csv", dtype=str).fillna("")

In [6]:
df_entity = pd.merge(df_entity_idx, df_node, how="left", left_on=["entity_id"], right_on=["node_id"])

In [28]:
drug_id_entity_dict = {}
drug_idx_id_dict = {}
count = 0
for index, row in df_entity.iterrows():
    if row["labels"] == "drug":
        drug_id_entity_dict[row["idx"]] = [row["entity_id"], row["name"], row["labels"]]
        drug_idx_id_dict[count] = row["idx"]
        count += 1

In [87]:
from sklearn.metrics.pairwise import cosine_similarity


In [22]:
len(drug_id_entity_dict)

5863

In [17]:
select_list = []
for key in drug_id_entity_dict.keys():
    select_list.append(node_embedding[int(key)])
select_embedding = np.array(select_list)

In [19]:
drug_similarity = cosine_similarity(select_embedding)

In [21]:
drug_similarity.shape

(5863, 5863)

In [81]:
result_dict = {}
for idx in range(drug_similarity.shape[0]):
    sort_list = sorted([[i, x] for i, x in enumerate(drug_similarity[idx])], key=lambda x: x[1], reverse=True)
    sort_list = list(filter(lambda x: x[0] != idx, sort_list))
    sort_list = list(map(lambda x: [drug_idx_id_dict[x[0]], x[1]], sort_list))[:3]
    result_dict[drug_idx_id_dict[idx]] = sort_list

In [82]:
drug_id_entity_dict["1829"]

['64466', '阿昔莫司胶囊', 'drug']

In [83]:
drug_id_entity_dict["11163"]

['64465', '苯扎贝特片', 'drug']

In [84]:
drug_list = []
drug_match_list = []
score_match_list = []
for key, value in result_dict.items():
    drug = drug_id_entity_dict[key][1]
    md_list = []
    ms_list = []
    for val in value:
        matched_drug = drug_id_entity_dict[val[0]][1]
        match_score = val[1]
        md_list.append(matched_drug)
        ms_list.append(match_score)
    drug_match_list.extend(md_list)
    score_match_list.extend(ms_list)
    drug_list.extend([drug] * len(md_list))

In [89]:
df_match = pd.DataFrame({
    "drug": drug_list,
    "matched_drug": drug_match_list,
    "score": score_match_list
}) # .to_csv("drug_similarity.csv", index=False)

In [92]:
df_match[:30]

,drug,matched_drug,score
0,醋酸泼尼松片,曲安西龙片,0.769136
1,醋酸泼尼松片,醋酸地塞米松片,0.742230
2,醋酸泼尼松片,地塞米松片,0.655462
3,三磷酸腺苷二钠片,荣心丸,0.759191
4,三磷酸腺苷二钠片,三磷酸腺苷二钠注射液,0.739641
5,三磷酸腺苷二钠片,心荣颗粒,0.677750
6,利培酮胶囊,富马酸喹硫平片,0.802860
7,利培酮胶囊,氯氮平片,0.776621
8,利培酮胶囊,氯氮平口腔崩解片,0.769843
9,氟哌啶醇片,氢溴酸加兰他敏分散片,0.721227


In [94]:
from py2neo import Graph 
session = Graph("neo4j://172.16.227.27:7687", auth=("neo4j", "123456"))

In [95]:
link_template = """
match (d1:drug {display: "氟哌啶醇注射液"})-[r]->(n:disease)<-[]-(d2:drug {display: "氟哌啶醇片"})
return d1.display as drug_name_1, n.display as link_node_name, labels(n)[0] as link_node_label, d2.display as drug_name_2
union
match (d1:drug {display: "氟哌啶醇注射液"})<-[]-(n:chemical)-[]->(d2:drug {display: "氟哌啶醇片"})
return d1.display as drug_name_1, n.display as link_node_name, labels(n)[0] as link_node_label, d2.display as drug_name_2
"""

result_list = session.run(link_template).data()
df_link = pd.DataFrame(result_list)

In [103]:
stats_dict = {}
for index, row in df_link.groupby(["link_node_label"])["link_node_name"].count().reset_index().iterrows():
    if row["link_node_label"] == "chemical":
        stats_dict["common_chemical_count"] = row["link_node_name"]
    if row["link_node_label"] == "disease":
        stats_dict["common_indication_count"] = row["link_node_name"]

In [104]:
stats_dict

{'common_chemical': 1, 'common_indication': 22}

In [115]:
indication_link_list = []
for index, row in df_link[df_link["link_node_label"] == "disease"].iterrows():
    indication_link_list.append("{}-[适应症]->{}<-[适应症]-{}".format(
        row["drug_name_1"], row["link_node_name"], row["drug_name_2"]
    ))

In [113]:
chemical_link_list = []
for index, row in df_link[df_link["link_node_label"] == "chemical"].iterrows():
    chemical_link_list.append("{}-[化合物]->{}<-[化合物]-{}".format(
        row["drug_name_1"], row["link_node_name"], row["drug_name_2"]
    ))

In [114]:
chemical_link_list

['氟哌啶醇注射液-[化合物]->haloperidol<-[化合物]-氟哌啶醇片']

In [124]:
link_template = """
match (d1:drug {{display: "{d1}"}})-[]->(n:disease)<-[]-(d2:drug {{display: "{d2}"}})
return d1.display as drug_name_1, n.display as link_node_name, labels(n)[0] as link_node_label, d2.display as drug_name_2
union
match (d1:drug {{display: "{d1}"}})<-[]-(n:chemical)-[]->(d2:drug {{display: "{d2}"}})
return d1.display as drug_name_1, n.display as link_node_name, labels(n)[0] as link_node_label, d2.display as drug_name_2
"""

common_chemical_count_list = []
common_indication_count_list = []
indication_links = []
chemical_links = []

for index, row in df_match.iterrows():
    link_query = link_template.format(d1=row["drug"], d2=row["matched_drug"])
    result_list = session.run(link_query).data()
    df_link = pd.DataFrame(result_list)
    chemical_count = 0
    indication_count = 0
    try:
        for index, row in df_link.groupby(["link_node_label"])["link_node_name"].count().reset_index().iterrows():
            if row["link_node_label"] == "chemical":
                chemical_count = row["link_node_name"]
            if row["link_node_label"] == "disease":
                indication_count = row["link_node_name"]
    except:
        pass
    
    common_chemical_count_list.append(chemical_count)
    common_indication_count_list.append(indication_count)
            
    indication_link_list = []
    try:
        for index, row in df_link[df_link["link_node_label"] == "disease"].iterrows():
            indication_link_list.append("{}-[适应症]->{}<-[适应症]-{}".format(
                row["drug_name_1"], row["link_node_name"], row["drug_name_2"]
            ))
    except:
        pass
    indication_links.append("\n".join(indication_link_list))
        
    chemical_link_list = []
    try:
        for index, row in df_link[df_link["link_node_label"] == "chemical"].iterrows():
            chemical_link_list.append("{}-[化合物]->{}<-[化合物]-{}".format(
                row["drug_name_1"], row["link_node_name"], row["drug_name_2"]
            ))
    except:
        pass
    chemical_links.append("\n".join(chemical_link_list))

In [128]:
df_match = df_match.assign(common_chemical_count=common_chemical_count_list)
df_match = df_match.assign(common_indication_count=common_indication_count_list)
chemical_links = [x.replace("\n", " | ") for x in chemical_links]
indication_links = [x.replace("\n", " | ") for x in indication_links]
df_match = df_match.assign(chemical_link=chemical_links)
df_match = df_match.assign(indication_link=indication_links)

In [129]:
df_match.to_csv("similar_drug_and_link.csv", index=False)